# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

# CODE

### Import necessary libraries

In [178]:
import time
import math
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix,lil_matrix
from sklearn.decomposition import TruncatedSVD


In [179]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [180]:
config = pd.read_csv('config.csv').iloc[5]

In [181]:
config

Unnamed: 0                                      0
id                                         276633
k                                               5
dataset_file              ./Behance_appreciate_1M
output_file     ./testing/output_test_svd_new.txt
Name: 5, dtype: object

In [182]:
user = int(config['id'])
k_value = int(config['k'])

### Read the Data

In [183]:
with open(config['dataset_file'], 'r') as inFile:
    appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [184]:
user_likes = dict()
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    if user_id not in user_likes:
        user_likes[user_id] = list()
    user_likes[user_id].append(item_id)

In [185]:
lis1 = []
lis2 = []
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1]) 
    lis1.append(user_id)
    lis2.append(item_id)
lis1=list(set(lis1))
lis2=list(set(lis2))

In [186]:
user_idx={}
arc=0
for i in lis1:
    user_idx[i]=arc
    arc+=1
item_idx={}
arc=0
for i in lis2:
    item_idx[i]=arc
    arc+=1
inv_user_idx={}
for idx in user_idx.keys():
    inv_user_idx[user_idx[idx]]=idx

In [187]:
data=lil_matrix((len(lis1),len(lis2)),dtype=float)

In [188]:
for line in appreciate_data:
    line = line.strip()
    user_id = int(line.split()[0])
    item_id = int(line.split()[1])
    data[user_idx[user_id],item_idx[item_id]]=1.0

In [189]:
del appreciate_data

### Use KNN after Collaborative Filtering to find nearest neighbors

In [190]:
# built data with less dimension
tt=TruncatedSVD(n_components=3000)
X=tt.fit_transform(data)

In [191]:
# export developed data
# with open('./testing/test_3000.pkl','wb') as f:
#     pickle.dump(X, f)

In [192]:
# import developed data
# dataset with reduced dimension can be imported from https://www.kaggle.com/architjain128/svd3000
# with open('./testing/test_3000.pkl','rb') as f:
#     X = pickle.load(f)

In [206]:
def diff_list(list1, list2):
    """ returns difference of two lists """
    return [x for x in list1 if x not in list2]

def cosine_similarity_score(list1, list2):
    """ returns cosine similarity score of two lists """
    val=0.0
    a=0.0
    b=0.0
    for i in range(len(list1)):
        val+=list1[i]*list2[i]
        a+=list1[i]*list1[i]
        b+=list2[i]*list2[i]
    try:
        return math.sqrt(max(0.0,1.0-((val*val)/(a*b))))
    except:
        return 1.0

def euclidean_distance_score(list1, list2):
    """ returns euclidean distance score of two lists """
    val=0.0
    for i in range(len(list1)):
        val+=(list1[i]-list2[i])*(list1[i]-list2[i])
    return math.sqrt(val)

def manhattan_distance(list1, list2):
    """ returns manhattan distance score of two lists """
    val=0.0
    for i in range(len(list1)):
        val+=abs(list1[i]-list2[i])
    return val

def ruzika_distace(list1, list2):
    """ returns ruzika distance score of two lists """
    val=0.0
    a=0.0
    for i in range(len(list1)):
        val+=abs(list1[i]-list2[i])
        a+=max(list1[i],list2[i])
    try :
        return 1.0 - val/a
    except:
        return 1.0
    
def wow_distace(list1, list2):
    """ returns ruzika distance score of two lists """
    val=0.0
    a=0.0
    b=0.0
    for i in range(len(list1)):
        val+=((list1[i]-list2[i])*(list1[i]-list2[i]) + abs(list1[i]-list2[i]))
    return math.sqrt(val)


In [222]:
def neighbors(user,k_value,X,distance_metric=euclidean_distance_score):
    """ returns an iterable object (like list or generator) """
    scores = {}
    for x in range(len(X)):
        x=inv_user_idx[x]
        if x != user:
            scores[x] = distance_metric(X[user_idx[user]],X[user_idx[x]])
    return sorted(scores.items(), key=lambda x:x[1], reverse=False)


In [223]:
scores=neighbors(user,k_value,X,wow_distace)

Viola


In [230]:
score=scores[:k_value]

[(2452817, 26.73653758871646),
 (1480429, 26.80415197919863),
 (1459742, 26.996489359810205),
 (3103074, 26.999197345309728),
 (1999429, 27.003505612370336)]

### Open the output file to write all the lines to the file

In [231]:
outFile = open(config['output_file'], 'w')

for n_user in score:
    user_id = n_user[0]
    for item_id in diff_list(user_likes[user_id],user_likes[user]):
        outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()